In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm
from tqdm import tqdm

# Dual Problem and Upper Bounds for American Options

<strong>Primal Problem</strong> (maximizing over stopping times)

\begin{equation*}
V_t=\sup_{\tau\in\mathcal{T}_{t, T}}\mathbb{E}^\mathbb{Q}\left[D_{t, \tau}F_{\tau}\big|\mathcal{F}_t\right]
\end{equation*}

<strong>Dual Problem</strong> (minimizing over martingales) [Rogers 2002; Haugh and Kogan 2004]

\begin{equation*}
V_t=\inf_{M\in\mathcal{M}_{t, 0}}\mathbb{E}^\mathbb{Q}\left[\sup_{t\leq s\leq T}\left(D_{t, s}F_s-M_s\right)\bigg|\mathcal{F}_t\right].
\end{equation*}

where $\mathcal{M}_{t, 0}$ denotes the set of all right-continuous martingales $(M_s, s\in[t, T])$ with $M_t=0$.
The optimal martingale $M^{\ast}$ is the martingale part of the Doob-Meyer decomposition of $(S_s-S_t)/D_{0, t}$, $t\leq s\leq T$, where $S$ is the Snell envelope of the discounted payoff, i.e., $S_t=\sup_{\tau\in\mathcal{T}_{t,T}}\mathbb{E}^{\mathbb{Q}}\left[D_{0,\tau}F_{\tau}\right]$.

In particular, any martingale $M_s$ with $M_0=0$ gives an upper bound for the price at time 0, $V_0$:

$$V_0 \leq\mathbb{E}^{\mathbb{Q}}\left[\sup_{0\leq s\leq T}\left(D_sF_s-M_s\right)\right].$$

## Section 1

Consider pricing a one-year Bermudan put option with monthly exercise, where the asset price process is assumed to follow geometric Brownian motion with $S_0=100$, $\sigma=0.2$, $r=0.1$, $q=0.02$, $K=100$ and exercise dates $t_1=\frac{1}{12}$, $t_2=\frac{2}{12}$, $\cdots$, $t_{12}=1$.

Simulate 100,000 paths and use the following martingale to find a upper bound.

(a). $M_t\equiv0$.

(b). $M_t$ is the discounted European put price with the same final maturity less the initial price.

For the reference, the price of the option is $5.152$.

In [ ]:
def blackscholes_mc(S=100, vol=0.2, r=0, q=0, ts=np.linspace(0, 1, 13), npaths=10):
    """Generate Monte-Carlo paths in Black-Scholes model.

    Parameters
    ----------
    S: scalar
        The spot price of the underlying security.
    vol: scalar
        The implied Black-Scholes volatility.
    r: scalar
        The annualized risk-free interest rate, continuously compounded.
    q: scalar
        The annualized continuous dividend yield.
    ts: array_like
        The time steps of the simualtion
    npaths: int
        the number of paths to simulate

    Returns
    -------
    paths: ndarray
        The Monte-Carlo paths.
    """
    nsteps = len(ts) - 1
    ts = np.asfarray(ts)[:, np.newaxis] # (nsteps+1, 1)
    W = np.cumsum(np.vstack((np.zeros((1, npaths), dtype=float), # (1, n_paths)
                             np.random.randn(nsteps, npaths) * np.sqrt(np.diff(ts, axis=0)))), # (nsteps, npaths) => (nsteps+1, npaths)
                  axis=0)
    paths = np.exp(-0.5*vol**2*ts + vol*W)*S*np.exp((r-q)*ts)
    return paths


def blackscholes_price(K, T, S, vol, r=0, q=0, callput='call'):
    """Compute the call/put option price in the Black-Scholes model

    Parameters
    ----------
    K: scalar or array_like
        The strike of the option.
    T: scalar or array_like
        The maturity of the option, expressed in years (e.g. 0.25 for 3-month and 2 for 2 years)
    S: scalar or array_like
        The current price of the underlying asset.
    vol: scalar or array_like
        The implied Black-Scholes volatility.
    r: scalar or array_like
        The annualized risk-free interest rate, continuously compounded.
    q: scalar or array_like
        The annualized continuous dividend yield.
    callput: str
        Must be either 'call' or 'put'.

    Returns
    -------
    price: scalar or array_like
        The price of the option.

    Examples
    --------
    >>> blackscholes_price(95, 0.25, 100, 0.2, r=0.05, callput='put')
    1.5342604771222823
    """
    F = S*np.exp((r-q)*T)
    v = np.sqrt(vol**2*T)
    d1 = np.log(F/K)/v + 0.5*v
    d2 = d1 - v
    try:
        opttype = {'call':1, 'put':-1}[callput.lower()]
    except:
        raise ValueError('The value of callput must be either "call" or "put".')
    price = opttype*(F*norm.cdf(opttype*d1)-K*norm.cdf(opttype*d2))*np.exp(-r*T)
    return price

In [ ]:
# Set up the parameters and the Monte Carlo Paths
np.random.seed(5400)
S0, vol, r, q, K = 100, 0.2, 0.1, 0.02, 100
ts = np.linspace(0, 1, 13)
npaths = 100000
paths = blackscholes_mc(S=S0, vol=vol, r=r, q=q, ts=ts, npaths=npaths)

# Generate the discounted payoffs from exercise of all paths at t from 1 to T
exer_func = lambda S: np.maximum(K-S, 0)
DsFs = exer_func(paths[1:])*np.exp(-r*ts[1:, np.newaxis])

In [ ]:
# Let the martingale be constant 0
M = 0

# For each path, get the maximal value of discounted payoff from exercise minus the martingale
# Get the expected value of maxima of all paths
upper_bound = np.mean(np.max(DsFs-M, axis=0))
print('The Upper Bound using Mt = 0 is {:.4f}'.format(upper_bound))

The Upper Bound using Mt = 0 is 8.6200


In [ ]:
# Get the Black-Scholes prices of all paths at t=0 to t=T-1
bs_price = blackscholes_price(K=100, T=(ts[-1]-ts[:-1])[:, np.newaxis], S=paths[:-1], vol=vol, r=r, q=q, callput='put')
# Discount Black-Scholes prices to values at t = 0
bs_price = bs_price * np.exp(-r*ts[:-1, np.newaxis])
# Get the payoffs from exercising at t = T
final_payoff = (exer_func(paths[-1])*np.exp(-r*ts[-1])).reshape(1, -1)
# Stack the payoffs at the end and Subtract the initial price to get the martingale
M = np.vstack((bs_price, final_payoff)) - bs_price[0]

# For each path, get the maximal value of discounted payoff from exercise minus the martingale
# Get the expected value of maxima of all paths
upper_bound = np.mean(np.max(DsFs-M[1:], axis=0))
print('The Upper Bound using discounted European put price minus the initial price is {:.4f}'.format(upper_bound))

The Upper Bound using discounted European put price minus the initial price is 5.3569


## Section 2

Optimal martingale

The optimal martingale that gives zero <em>duality gap</em> is the martingale component of the discounted true value process (known as Snell envelope) $\frac{V_t}{D_{0,t}}$, following the optimal exercise strategy.

To get a good upper bound, we first find a value process that is close to the true value process, and then extract its martingale component. This approximate value process can be obtained from a functional approximation such as regression, or from an exercise strategy (stopping time).

In general, for a stochastic process $\{U_n\}_{n\geq0}$, to extract its martingale component, we only need to let $M_0=0$, and then

$$M_{n+1}-M_n=U_{n+1}-\mathbb{E}\left[U_{n+1}\big|\mathcal{F}_n\right],\quad n=0,1,\cdots.$$

As a demonstration, we show below the code that calculates the upper bound estimate of the price of Bermudan put with monthly exercises as in Section 1. The optimal exercise frontier is given for each exercise date (from a PDE solver), i.e. if the underlying price falls below Bs[i] at time ts[i], then we should exercise the option, otherwise we should continue.

In [ ]:
Bs = np.array([87.5297, 87.7804, 88.0589, 88.3694, 88.724, 89.1309, 89.6067, 90.1723, 90.8653, 91.7451, 92.956, 94.8524, 100])

def exer_or_cont(i, S):
    # exer if True else cont
    return S <= Bs[i]

In [ ]:
np.random.seed(5400)
S0, K, vol, r, q = 100, 100, 0.2, 0.1, 0.02
ts = np.linspace(0, 1, 13)
exer_func = lambda S: np.maximum(K-S, 0)

# nested simulation from time t_i when stock price is S
def nested_mc(S, vol, r, q, i, ts, nnested):
    nested_paths = np.full(nnested, S, dtype=float)
    tot_payoff = 0
    for j in range(i+1, len(ts)):
        dt = ts[j] - ts[j-1]
        dW = np.random.randn(len(nested_paths))*np.sqrt(dt)        # Brownian increment
        nested_paths = nested_paths*np.exp((r-q)*dt)*np.exp(-0.5*vol**2*dt + vol*dW)
        exer_vals =  exer_func(nested_paths)
        if j < len(ts)-1:
            ind = exer_or_cont(j, nested_paths)                    # identify the paths that need exercise
            tot_payoff += np.sum(exer_vals[ind])*np.exp(-r*ts[j])
            nested_paths = nested_paths[~ind]                      # remove exercised paths
            if len(nested_paths) == 0:                             # if exercised for all paths, stop
                break
        else:
            tot_payoff +=  np.sum(exer_vals)*np.exp(-r*ts[j])
    return tot_payoff/nnested                                      # taking average of paths

# Simulate independent paths and exercise the option acoording to this strategy
V0 = nested_mc(S0, vol, r, q, 0, ts, 1000000)

# Upper bound by Andersen-Broadie algorithm
npaths = 500                                                       # number of paths in the second independent run
nnested = 1000                                                     # number of paths used in nested simulation
paths = blackscholes_mc(S=S0, vol=vol, r=r, q=q, ts=ts, npaths=npaths)
V = np.full(paths.shape, np.nan, dtype=float)                   # discounted value process V_i (discounted to time zero)
EV = np.full(paths.shape, np.nan, dtype=float)                  # Conditional expectation E[V_{i+1}|F_i], or continuation value
V[0] = V0                                                          # Initial value from lower bound simulation
EV[0] = V0                                                         # at time 0, option value = continuation value
for i in tqdm(range(1, len(ts)-1)):
    exer_vals =  exer_func(paths[i])
    ind = exer_or_cont(i, paths[i])                                # True for exercise False for continue
    for j in np.nonzero(ind)[0]:
        V[i, j] = exer_vals[j]*np.exp(-r*ts[i])                    # if exercised, V[i,j] = exercise value
        EV[i,j] = nested_mc(paths[i, j], vol, r, q, i, ts, nnested) # launch nested simulation to estimate E[V_{i+1}|F_i]
    for j in np.nonzero(~ind)[0]:
        V[i,j] = nested_mc(paths[i, j], vol, r, q, i, ts, nnested)  # if continue, use nested simulation to estimate V[i, j]
        EV[i, j] = V[i, j]                                         # E[V_{i+1}|F_i] = V_i
V[-1] = exer_func(paths[-1])*np.exp(-r*ts[-1])                     # values at final maturity
hedges = np.zeros(paths.shape, dtype=float)
hedges[1:] = np.cumsum(V[1:]-EV[:-1], axis=0)                      # martinglae increment V_{i+1}-E[V_{i+1}|F_i]

print('Primal Price (Lower Bound) = {:.4f}'.format(V0))
print('Dual Price   (Upper Bound) = {:.4f}'.format(np.mean(np.amax(exer_func(paths[1:])*np.exp(-r*ts[1:, np.newaxis])-hedges[1:], axis=0))))

100%|██████████| 11/11 [00:05<00:00,  2.02it/s]

Primal Price (Lower Bound) = 5.1514
Dual Price   (Upper Bound) = 5.1608


Since the strategy is optimal, we see that both lower bound and upper bound prices converge to the true value.

However, in general, we may only get a sub-optimal strategy (e.g. from Longstaff-Schwartz algorithm). But with this sub-optimal strategy, we can build the (discounted) value process and then extract its martingale component to obtain an upper bound. Intuitively speaking, the closer this sub-optimal strategy is to the optimal one, the tighter the corresponding upper bound.

Consider pricing the Bermudan put option as in Section 1 by using the constant $1.0$ and the Black-Scholes price of a European put option with volatility $0.2$ and maturity $T-t_n$ as the two basis functions at time $t_n$. Use the Longstaff-Schwartz algorithm to build an exercise strategy with these basis functions and then compute the corresponding upper bound.

In [ ]:
np.random.seed(5400)
S0, K, vol, r, q = 100, 100, 0.2, 0.1, 0.02
ts = np.linspace(0, 1, 13)

npaths = 100000
paths = blackscholes_mc(S=S0, vol=vol, r=r, q=q, ts=ts, npaths=npaths)
betas_LS = np.full((len(ts), 2), np.nan, dtype=float)
payoff = np.maximum(K-paths[-1], 0)
for i in tqdm(range(len(ts)-2, 0, -1)):
    discount = np.exp(-r*(ts[i+1]-ts[i]))
    payoff = payoff*discount
    Z = blackscholes_price(K, ts[-1]-ts[i], paths[i], 0.2, r, q, callput='put')
    A = np.vstack((np.ones_like(Z), Z)).T
    betas_LS[i] = np.linalg.lstsq(A, payoff, rcond=None)[0]    # regression to estimate continuation values
    contval = betas_LS[i, 0]+betas_LS[i, 1]*Z
    exerval = np.maximum(K-paths[i], 0)
    # identify the paths where we should exercise
    ind = exerval > contval
    payoff[ind] = exerval[ind]                     # update payoff on exercised paths
betas_LS                                           # No regression needed at first and last time steps, return NaN

100%|██████████| 11/11 [00:00<00:00, 14.86it/s]


array([[        nan,         nan],
       [-0.51648977,  1.30692294],
       [-0.46315555,  1.28765299],
       [-0.39635895,  1.26059044],
       [-0.2695087 ,  1.21813049],
       [-0.20345165,  1.19510851],
       [-0.13763973,  1.16287821],
       [-0.0685234 ,  1.1276664 ],
       [-0.03460999,  1.09958488],
       [-0.00142711,  1.06398158],
       [ 0.00235764,  1.03423006],
       [-0.00248291,  1.00262258],
       [        nan,         nan]])

In [ ]:
np.random.seed(2024)

# nested simulation from time t_i when stock price is S
def nested_mc_bs(S, vol, r, q, i, ts, nnested):
    nested_paths = np.full(nnested, S, dtype=float)
    tot_payoff = 0
    for j in range(i+1, len(ts)):
        dt = ts[j] - ts[j-1]
        dW = np.random.randn(len(nested_paths))*np.sqrt(dt)        # Brownian increment
        nested_paths = nested_paths*np.exp((r-q)*dt)*np.exp(-0.5*vol**2*dt + vol*dW)
        exer_vals =  exer_func(nested_paths)
        if j < len(ts)-1:
            Z = blackscholes_price(K, ts[-1]-ts[j], nested_paths, vol, r, q, callput='put')
            cont_vals = betas_LS[j, 0]+betas_LS[j, 1]*Z
            ind = exer_vals > cont_vals                            # identify the paths that need exercise
            tot_payoff += np.sum(exer_vals[ind])*np.exp(-r*ts[j])
            nested_paths = nested_paths[~ind]                      # remove exercised paths
            if len(nested_paths) == 0:                             # if exercised for all paths, stop
                break
        else:
            tot_payoff +=  np.sum(exer_vals)*np.exp(-r*ts[j])
    return tot_payoff/nnested                                      # taking average of paths

# Simulate independent paths and exercise the option acoording to this strategy
V0 = nested_mc_bs(S0, vol, r, q, 0, ts, 1000000)

# Upper bound by Andersen-Broadie algorithm
npaths = 500                                                       # number of paths in the second independent run
nnested = 1000                                                     # number of paths used in nested simulation
paths = blackscholes_mc(S=S0, vol=vol, r=r, q=q, ts=ts, npaths=npaths)
V = np.full(paths.shape, np.nan, dtype=float)                   # discounted value process V_i (discounted to time zero)
EV = np.full(paths.shape, np.nan, dtype=float)                  # Conditional expectation E[V_{i+1}|F_i], or continuation value
V[0] = V0                                                          # Initial value from lower bound simulation
EV[0] = V0                                                         # at time 0, option value = continuation value
for i in tqdm(range(1, len(ts)-1)):
    exer_vals = exer_func(paths[i])
    Z = blackscholes_price(K, ts[-1]-ts[i], paths[i], vol, r, q, callput='put')
    cont_vals = betas_LS[i, 0]+betas_LS[i, 1]*Z
    ind = exer_vals > cont_vals                                    # True for exercise False for continue
    for j in np.nonzero(ind)[0]:
        V[i, j] = exer_vals[j]*np.exp(-r*ts[i])                    # if exercised, V[i,j] = exercise value
        EV[i,j] = nested_mc_bs(paths[i, j], vol, r, q, i, ts, nnested) # launch nested simulation to estimate E[V_{i+1}|F_i]
    for j in np.nonzero(~ind)[0]:
        V[i,j] = nested_mc_bs(paths[i, j], vol, r, q, i, ts, nnested)  # if continue, use nested simulation to estimate V[i, j]
        EV[i, j] = V[i, j]                                         # E[V_{i+1}|F_i] = V_i
V[-1] = exer_func(paths[-1])*np.exp(-r*ts[-1])                     # values at final maturity
hedges = np.zeros(paths.shape, dtype=float)
hedges[1:] = np.cumsum(V[1:]-EV[:-1], axis=0)                      # martinglae increment V_{i+1}-E[V_{i+1}|F_i]
print('\n')
print('Upper Bound (1 & BS price) = {:.4f}'.format(np.mean(np.amax(exer_func(paths[1:])*np.exp(-r*ts[1:, np.newaxis])-hedges[1:], axis=0))))

100%|██████████| 11/11 [00:16<00:00,  1.52s/it]



Upper Bound (1 & BS price) = 5.1645


## Section 3

(An alternative algorithm to estimate the upper bound)</b> The primal algorithm (Longstaff-Schwartz or Tsitsiklis-Van Roy) provides a function approximation (via regression) of the continuation value $C_n$ at $t_n$, hence of the value function $V_n = \max(C_n,F_n)$ where $F_n$ is the exercise value. We can use this functional approximation of the value function to write an alternative algorithm in order to estimate the upper bound: at each time $t_n$, use the functional approximation of $V_n$, and run the nested simulation to estimate the conditional expectation $\mathbb{E}\left[V_{n+1}\big|\mathcal{F}_n\right]$ (using the functional approximation of $V_{n+1}$). Note that this nested simulation only needs to be run for one time step.

For the same Bermudan put option and same basis functions, use TVR algorithm to build the value process and then extract its martingale component using the alternative algorithm explained above to estimate the upper bound price.

In [ ]:
np.random.seed(5400)
S0, K, vol, r, q = 100, 100, 0.2, 0.1, 0.02
ts = np.linspace(0, 1, 13)

npaths = 100000
paths = blackscholes_mc(S=S0, vol=vol, r=r, q=q, ts=ts, npaths=npaths)
betas_TVR = np.full((len(ts), 2), np.nan, dtype=float)
V = np.maximum(K-paths[-1], 0)
for i in range(len(ts)-2, 0, -1):
    discount = np.exp(-r*(ts[i+1]-ts[i]))
    Z = blackscholes_price(K, ts[-1]-ts[i], paths[i], vol, r, q, callput='put')
    A = np.vstack((np.ones_like(Z), Z)).T
    betas_TVR[i] = np.linalg.lstsq(A, V*discount, rcond=None)[0]      # regression to estimate continuation values
    contval = betas_TVR[i, 0]+betas_TVR[i, 1]*Z
    exerval = np.maximum(K-paths[i], 0)
    V = np.maximum(exerval, contval)                      # compute values
betas_TVR                                                 # No regression needed at first and last time steps, return NaN

array([[            nan,             nan],
       [-5.19310302e-01,  1.33489359e+00],
       [-4.16109641e-01,  1.30292903e+00],
       [-3.08496442e-01,  1.26319714e+00],
       [-2.06981527e-01,  1.22636101e+00],
       [-1.42015666e-01,  1.20088028e+00],
       [-7.99598280e-02,  1.16622183e+00],
       [-2.97276411e-02,  1.13086524e+00],
       [-1.05593731e-02,  1.09932413e+00],
       [ 6.79765209e-03,  1.06360783e+00],
       [ 5.24278216e-04,  1.03457529e+00],
       [-2.48290688e-03,  1.00262258e+00],
       [            nan,             nan]])

In [ ]:
np.random.seed(2024)

# nested simulation from time t_i when stock prices are Ss
def nested_mc_alt(Ss, vol, r, q, i, ts, nnested):
    dt = ts[i+1] - ts[i]
    dW = np.random.randn(len(Ss), nnested)*np.sqrt(dt)
    nested_paths = Ss[:, np.newaxis] * np.exp((r-q)*dt) * np.exp(-0.5*vol**2*dt + vol*dW)
    exer_vals =  exer_func(nested_paths)
    if i < len(ts)-2:
        Z = blackscholes_price(K, ts[-1]-ts[i+1], nested_paths, vol, r, q, callput='put')
        cont_vals = betas_TVR[i+1, 0]+betas_TVR[i+1, 1]*Z
        V = np.maximum(exer_vals, cont_vals) * np.exp(-r*ts[i+1])
    else:
        V = exer_vals * np.exp(-r*ts[i+1])
    return np.mean(V, axis = 1)

# Simulate independent paths and exercise the option acoording to this strategy
npaths = 500                                                       # number of paths in the second independent run
V0 = nested_mc_alt(S0*np.ones(npaths), vol, r, q, 0, ts, 10000)

# Upper bound
nnested = 10000                                                     # number of paths used in nested simulation
paths = blackscholes_mc(S=S0, vol=vol, r=r, q=q, ts=ts, npaths=npaths)
V = np.full(paths.shape, np.nan, dtype=float)                   # discounted value process V_i (discounted to time zero)
EV = np.full(paths.shape, np.nan, dtype=float)                  # Conditional expectation E[V_{i+1}|F_i], or continuation value
V[0] = V0                                                          # Initial value from lower bound simulation
EV[0] = V0                                                         # at time 0, option value = continuation value
for i in tqdm(range(1, len(ts)-1)):
    exer_vals = exer_func(paths[i])                                # get the exercised value
    Z = blackscholes_price(K, ts[-1]-ts[i], paths[i], vol, r, q, callput='put')
    cont_vals = betas_TVR[i, 0]+betas_TVR[i, 1]*Z                  # get the continuation value from the function approximation
    V[i] = np.maximum(exer_vals, cont_vals) * np.exp(-r*ts[i])     # use the discounted maximum of these two as the true V_i process
    EV[i] = nested_mc_alt(paths[i], vol, r, q, i, ts, nnested)     # launch nested simulation to estimate E[V_{i+1}|F_i]
V[-1] = exer_func(paths[-1])*np.exp(-r*ts[-1])                     # values at final maturity
hedges = np.zeros(paths.shape, dtype=float)
hedges[1:] = np.cumsum(V[1:]-EV[:-1], axis=0)                      # martingale increment V_{i+1}-E[V_{i+1}|F_i]
print('\n')
print('Upper Bound (1 & BS price) = {:.4f}'.format(np.mean(np.amax(exer_func(paths[1:])*np.exp(-r*ts[1:, np.newaxis])-hedges[1:], axis=0))))

100%|██████████| 11/11 [00:13<00:00,  1.25s/it]



Upper Bound (1 & BS price) = 5.1606


## Section 4

The upper bound estimator can be used to check if the basis functions used in regression are adequate. A tight duality gap between the lower bound and upper bound is a confirmation that the linear span of the selected basis functions indeed gives a good approximation to the continuation value. We now illustrate this with the pricing of Bermudan-Asian call options.

Assume that all the parameters stay the same.

Reuse the exercise strategy derived from the basis functions (1, BlackScholes) by Longstaff-Schwartz with $\bar{\sigma}=0.1$ to estimate the lower and upper bound.  Note that the lower bound should be obtained by running new independent paths stopped according to the strategy from the Longstaff-Schwartz algorithm.

As an alternative set of basis functions at time $t_n$, use
$$1, A_{t_n}, S_{t_n}, A_{t_n}^2, S_{t_n}^2, A_{t_n}S_{t_n}.$$
in the Longstaff-Schwartz algorithm and then give the corresponding lower bound and upper bound.

Compare the numerical duality gaps obtained. Which set of basis functions does a better job at estimating the optimal exercise strategy?

In [ ]:
np.random.seed(5400)

# Longstaff-Schwartz
S0, K, vol, r, q = 100, 100, 0.2, 0, 0
ts = np.linspace(0, 1, 13)
npaths = 100000
pathsS = blackscholes_mc(S0, vol, r, q, ts=ts, npaths=npaths)                                  # stock prices
pathsA = np.vstack((pathsS[0], np.cumsum(pathsS[1:], axis=0)/np.arange(1, 13)[:, np.newaxis])) # running averages
vol_ = 0.1
payoff = np.maximum(pathsA[-1]-K, 0)
betas_AC = np.full((len(ts), 2), np.nan, dtype=float)
for i in range(len(ts)-2, 0, -1):
    discount = np.exp(-r*(ts[i+1]-ts[i]))
    payoff = payoff*discount
    Z = (i*pathsA[i]+(12-i)*pathsS[i])/12
    callZ = blackscholes_price(K, ts[-1]-ts[i], Z, vol_, callput='call')
    A = np.vstack((np.ones_like(Z), callZ)).T
    betas_AC[i] = np.linalg.lstsq(A, payoff, rcond=None)[0]           # regression to estimate continuation values
    contval = betas_AC[i, 0]+betas_AC[i, 1]*callZ
    exerval = np.maximum(pathsA[i]-K, 0)
    ind = exerval > contval                               # identify the paths where we should exercise
    payoff[ind] = exerval[ind]
betas_AC

array([[        nan,         nan],
       [ 0.22682868,  1.1322062 ],
       [-0.03140532,  1.1085478 ],
       [-0.19686484,  1.08965054],
       [-0.29162769,  1.07291482],
       [-0.33703702,  1.0560594 ],
       [-0.37144058,  1.04590732],
       [-0.38766094,  1.04229595],
       [-0.35716887,  1.03361323],
       [-0.31663984,  1.02538663],
       [-0.25546118,  1.01776191],
       [-0.16944658,  1.00803082],
       [        nan,         nan]])

In [ ]:
np.random.seed(2024)
exer_func_call = lambda S: np.maximum(S-K, 0)

# nested simulation from time t_i when stock price is S
def nested_mc_ac(S, A, vol, vol_, r, q, i, ts, nnested):
    nested_S = np.full(nnested, S, dtype=float)
    nested_A = np.full(nnested, A, dtype=float)
    tot_payoff = 0
    for j in range(i+1, len(ts)):
        dt = ts[j] - ts[j-1]
        dW = np.random.randn(len(nested_S))*np.sqrt(dt)        # Brownian increment
        nested_S = nested_S*np.exp((r-q)*dt)*np.exp(-0.5*vol**2*dt + vol*dW) # update the stock price
        if j == 1:
            nested_A = nested_S
        else:
            nested_A = (nested_A*(j-1)+nested_S)/j                # update the moving average
        exer_vals =  exer_func_call(nested_A)
        if j < len(ts)-1:
            Z = (j*nested_A+(12-j)*nested_S)/12                   # get the input for black-scholes
            callZ = blackscholes_price(K, ts[-1]-ts[j], Z, vol_, r, q, callput='call') # get the black-scholes price
            cont_vals = betas_AC[j, 0]+betas_AC[j, 1]*callZ
            ind = exer_vals > cont_vals                            # identify the paths that need exercise
            tot_payoff += np.sum(exer_vals[ind])*np.exp(-r*ts[j])
            nested_S = nested_S[~ind]                              # remove exercised paths
            nested_A = nested_A[~ind]
            if len(nested_S) == 0:                                 # if exercised for all paths, stop
                break
        else:
            tot_payoff +=  np.sum(exer_vals)*np.exp(-r*ts[j])
    return tot_payoff/nnested                                      # taking average of paths

# Simulate independent paths and exercise the option acoording to this strategy
V0 = nested_mc_ac(S0, S0, vol, vol_, r, q, 0, ts, 1000000)

# Upper bound by Andersen-Broadie algorithm
npaths = 500                                                       # number of paths in the second independent run
nnested = 1000                                                     # number of paths used in nested simulation
pathsS = blackscholes_mc(S=S0, vol=vol, r=r, q=q, ts=ts, npaths=npaths)                  # stock prices
pathsA = np.vstack((pathsS[0], np.cumsum(pathsS[1:], axis=0)/np.arange(1, 13)[:, np.newaxis])) # running averages
V = np.full(pathsS.shape, np.nan, dtype=float)                   # discounted value process V_i (discounted to time zero)
EV = np.full(pathsS.shape, np.nan, dtype=float)                  # Conditional expectation E[V_{i+1}|F_i], or continuation value
V[0] = V0                                                          # Initial value from lower bound simulation
EV[0] = V0                                                         # at time 0, option value = continuation value
for i in tqdm(range(1, len(ts)-1)):
    exer_vals = exer_func_call(pathsA[i])
    Z = (i*pathsA[i]+(12-i)*pathsS[i])/12
    callZ = blackscholes_price(K, ts[-1]-ts[i], Z, vol_, callput='call')
    cont_vals = betas_AC[i, 0]+betas_AC[i, 1]*callZ
    ind = exer_vals > cont_vals                                    # True for exercise False for continue
    for j in np.nonzero(ind)[0]:
        V[i, j] = exer_vals[j]*np.exp(-r*ts[i])                    # if exercised, V[i,j] = exercise value
        EV[i,j] = nested_mc_ac(pathsS[i, j], pathsA[i, j], vol, vol_, r, q, i, ts, nnested) # launch nested simulation to estimate E[V_{i+1}|F_i]
    for j in np.nonzero(~ind)[0]:
        V[i,j] = nested_mc_ac(pathsS[i, j], pathsA[i, j], vol, vol_, r, q, i, ts, nnested)  # if continue, use nested simulation to estimate V[i, j]
        EV[i, j] = V[i, j]                                         # E[V_{i+1}|F_i] = V_i
V[-1] = exer_func_call(pathsA[-1])*np.exp(-r*ts[-1])                     # values at final maturity
hedges = np.zeros(pathsS.shape, dtype=float)
hedges[1:] = np.cumsum(V[1:]-EV[:-1], axis=0)                      # martinglae increment V_{i+1}-E[V_{i+1}|F_i]
print('\n')
lb_ac = V0
ub_ac = np.mean(np.amax(exer_func_call(pathsA[1:])*np.exp(-r*ts[1:, np.newaxis])-hedges[1:], axis=0))
dg_ac = ub_ac - lb_ac
print('Bermudan-Asian Lower Bound (1 & BS price) = {:.4f}'.format(lb_ac))
print('Bermudan-Asian Upper Bound (1 & BS price) = {:.4f}'.format(ub_ac))
print('Bermudan-Asian Duality Gap (1 & BS price) = {:.4f}'.format(dg_ac))

100%|██████████| 11/11 [00:13<00:00,  1.21s/it]



Bermudan-Asian Lower Bound (1 & BS price) = 5.2973
Bermudan-Asian Upper Bound (1 & BS price) = 5.3926
Bermudan-Asian Duality Gap (1 & BS price) = 0.0953


In [ ]:
np.random.seed(5400)

# Longstaff-Schwartz
S0, K, vol, r, q = 100, 100, 0.2, 0, 0
ts = np.linspace(0, 1, 13)
npaths = 100000
pathsS = blackscholes_mc(S0, vol, r, q, ts=ts, npaths=npaths)                                  # stock prices
pathsA = np.vstack((pathsS[0], np.cumsum(pathsS[1:], axis=0)/np.arange(1, 13)[:, np.newaxis])) # running averages
vol_ = 0.1
payoff = np.maximum(pathsA[-1]-K, 0)
betas_PR = np.full((len(ts), 6), np.nan, dtype=float)
for i in range(len(ts)-2, 0, -1):
    discount = np.exp(-r*(ts[i+1]-ts[i]))
    payoff = payoff*discount
    A = np.vstack((np.ones_like(pathsS[i]), pathsA[i], pathsS[i], pathsA[i]**2, pathsS[i]**2, pathsA[i]*pathsS[i])).T
    betas_PR[i] = np.linalg.lstsq(A, payoff, rcond=None)[0]           # regression to estimate continuation values
    contval = A @ betas_PR[i]
    exerval = np.maximum(pathsA[i]-K, 0)
    ind = exerval > contval                               # identify the paths where we should exercise
    payoff[ind] = exerval[ind]
betas_PR

array([[            nan,             nan,             nan,
                    nan,             nan,             nan],
       [ 1.61925838e+02, -1.86139262e+00, -1.86139262e+00,
         7.15628425e-03,  7.15628425e-03,  7.15628325e-03],
       [ 1.58931518e+02, -1.59627677e+00, -2.06359138e+00,
         3.95971743e-03,  7.73528817e-03,  9.40175043e-03],
       [ 1.54298985e+02, -1.92744402e+00, -1.63477253e+00,
         9.61291272e-03,  8.98367257e-03,  1.94976964e-03],
       [ 1.49135766e+02, -2.21169150e+00, -1.24044483e+00,
         1.19798210e-02,  7.43938891e-03,  5.15450549e-04],
       [ 1.43654891e+02, -2.43749351e+00, -8.95252219e-01,
         1.38043845e-02,  5.98955541e-03, -5.26689183e-04],
       [ 1.38102584e+02, -2.57257430e+00, -6.40717892e-01,
         1.53782478e-02,  5.24422561e-03, -2.01457736e-03],
       [ 1.32160779e+02, -2.57137279e+00, -5.16354967e-01,
         1.56094704e-02,  4.53082932e-03, -2.21150519e-03],
       [ 1.26429602e+02, -2.56913475e+00, -3.959

In [ ]:
np.random.seed(2024)
exer_func_call = lambda S: np.maximum(S-K, 0)

# nested simulation from time t_i when stock price is S
def nested_mc_pr(S, A, vol, r, q, i, ts, nnested):
    nested_S = np.full(nnested, S, dtype=float)
    nested_A = np.full(nnested, A, dtype=float)
    tot_payoff = 0
    for j in range(i+1, len(ts)):
        dt = ts[j] - ts[j-1]
        dW = np.random.randn(len(nested_S))*np.sqrt(dt)        # Brownian increment
        nested_S = nested_S*np.exp((r-q)*dt)*np.exp(-0.5*vol**2*dt + vol*dW) # update the stock price
        if j == 1:
            nested_A = nested_S
        else:
            nested_A = (nested_A*(j-1)+nested_S)/j                # update the moving average
        exer_vals =  exer_func_call(nested_A)
        if j < len(ts)-1:
            A = np.vstack((np.ones_like(nested_A), nested_A, nested_S, nested_A**2, nested_S**2, nested_A*nested_S)).T
            cont_vals = A @ betas_PR[i]
            ind = exer_vals > cont_vals                            # identify the paths that need exercise
            tot_payoff += np.sum(exer_vals[ind])*np.exp(-r*ts[j])
            nested_S = nested_S[~ind]                              # remove exercised paths
            nested_A = nested_A[~ind]
            if len(nested_S) == 0:                                 # if exercised for all paths, stop
                break
        else:
            tot_payoff +=  np.sum(exer_vals)*np.exp(-r*ts[j])
    return tot_payoff/nnested                                      # taking average of paths

# Simulate independent paths and exercise the option acoording to this strategy
V0 = nested_mc_pr(S0, S0, vol, r, q, 0, ts, 1000000)

# Upper bound by Andersen-Broadie algorithm
npaths = 500                                                       # number of paths in the second independent run
nnested = 1000                                                     # number of paths used in nested simulation
pathsS = blackscholes_mc(S=S0, vol=vol, r=r, q=q, ts=ts, npaths=npaths)                  # stock prices
pathsA = np.vstack((pathsS[0], np.cumsum(pathsS[1:], axis=0)/np.arange(1, 13)[:, np.newaxis])) # running averages
V = np.full(pathsS.shape, np.nan, dtype=float)                   # discounted value process V_i (discounted to time zero)
EV = np.full(pathsS.shape, np.nan, dtype=float)                  # Conditional expectation E[V_{i+1}|F_i], or continuation value
V[0] = V0                                                          # Initial value from lower bound simulation
EV[0] = V0                                                         # at time 0, option value = continuation value
for i in tqdm(range(1, len(ts)-1)):
    exer_vals = exer_func_call(pathsA[i])
    A = np.vstack((np.ones_like(pathsS[i]), pathsA[i], pathsS[i], pathsA[i]**2, pathsS[i]**2, pathsA[i]*pathsS[i])).T
    cont_vals = A @ betas_PR[i]
    ind = exer_vals > cont_vals                                    # True for exercise False for continue
    for j in np.nonzero(ind)[0]:
        V[i, j] = exer_vals[j]*np.exp(-r*ts[i])                    # if exercised, V[i,j] = exercise value
        EV[i,j] = nested_mc_pr(pathsS[i, j], pathsA[i, j], vol, r, q, i, ts, nnested) # launch nested simulation to estimate E[V_{i+1}|F_i]
    for j in np.nonzero(~ind)[0]:
        V[i,j] = nested_mc_pr(pathsS[i, j], pathsA[i, j], vol, r, q, i, ts, nnested)  # if continue, use nested simulation to estimate V[i, j]
        EV[i, j] = V[i, j]                                         # E[V_{i+1}|F_i] = V_i
V[-1] = exer_func_call(pathsA[-1])*np.exp(-r*ts[-1])                     # values at final maturity
hedges = np.zeros(pathsS.shape, dtype=float)
hedges[1:] = np.cumsum(V[1:]-EV[:-1], axis=0)                      # martinglae increment V_{i+1}-E[V_{i+1}|F_i]
print('\n')
lb_pr = V0
ub_pr = np.mean(np.amax(exer_func_call(pathsA[1:])*np.exp(-r*ts[1:, np.newaxis])-hedges[1:], axis=0))
dg_pr = ub_pr - lb_pr
print('Bermudan-Asian Lower Bound (poly-regr) = {:.4f}'.format(lb_pr))
print('Bermudan-Asian Upper Bound (poly-regr) = {:.4f}'.format(ub_pr))
print('Bermudan-Asian Duality Gap (poly-regr) = {:.4f}'.format(dg_pr))

100%|██████████| 11/11 [00:05<00:00,  2.13it/s]



Bermudan-Asian Lower Bound (poly-regr) = 4.8855
Bermudan-Asian Upper Bound (poly-regr) = 5.3256
Bermudan-Asian Duality Gap (poly-regr) = 0.4402


From our simulation, the Duality Gap from constant 1 and Black-Scholes price is far less than the Duality gap from polynomial regression. Thus, the pair of constant 1 and Black-Scholes price as basis functions is more effective at estimating the optimal strategy.